# Practicals for lecture 1.1

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vigji/python-cimec/blob/main/practicals/Practicals_1.2.ipynb)

## Introduction to `pandas`

Create and index dataframes

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

In [ ]:
string_data = ['cabbage', 'artichoke', 'banana', 'avocado']
int_data = [1, 2, 3, 4]
float_data = [0.1, 0.2, 0.3, 0.4]

# Put together the data above in a DataFrame.



In [ ]:
# Using indexing, select the column of the dataframe containing the strings


In [ ]:
# Using indexing, select the first row of the dataframe


In [ ]:
# Using indexing, select the raws of the dataframe so that the fload data is greater than 0.2


#### Methods of `pandas` dataframes

In [ ]:
# Here we create a fake dataframe containing the results of an psychological test with 30 subjects.
# Subjects can be left-handed or right-handed.
# The test has 2 measures (reaction time - RT, and accuracy)
def create_data_df():
    np.seed(42)
    subject_ability = np.random.uniform(0, 1, 30)
    df = pd.DataFrame({'subject': [f"subject_{i}" for i in range(30)],
                       'handedness': np.random.choice(['left', 'right'], 30),
                       'RT': subject_ability*100 + np.random.uniform(0, 50, 30),
                       'accuracy': subject_ability + np.random.normal(0.8, 0.1, 30)})
df

In [ ]:
# sort the dataframe by RT:


In [ ]:
# Compute the mean and standard deviation of the RT and accuracy across:


In [ ]:
# Use indexing to select the RT of the left-handed subjects, and compute its 90% percentile:


In [ ]:
# Now use the percentile so select the accuracy for left-handed subjects with RT above the 90% percentile:


In [ ]:
# Create a scatter plot of RT vs accuracy for the right-handed subjects:
